# Code upgrade summary

We extend the changes from <a href = "http://www.github.com/dsirajud/IPython-notebooks/DECSKS-16 -- 1D1V_one_species_differentiating_the_application_of_HOC/DECSKS-16 -- 1D1V_one_species_differentiating_the_application_of_HOC.ipynb">DECSKS-16</a> which was for one species evolution to the code update from <a href = "http://www.github.com/dsirajud/IPython-notebook/DECSKS-14 -- Two_species_Vlasov_simulations/DECSKS-14 -- Two_species_Vlasov_simulations.ipynb">DECSKS-14</a>. Thus, the code upgrade in this folder represents a 1D1V DECSKS-2.0 Vlasov-Poisson two species evolution where it is possible to choose high order correction methods on each phase space variable x, vx, rather than having both be the same.

This change uncovered a resilience in the Fourier methods that is not available in finite difference methods. That is, random densities cannot be chosen if periodic BCs are used. This is obvious, i.e. we require $\int \rho d^3x = 0$, which is seen below to be the only way that periodic BCs on $\phi$ can be fulfilled for two boundaries $\partial S_1$ and $\partial S_2$: 

$$\int d^3x [\Delta \phi = -\rho ] \Rightarrow \partial_n \phi |_{\partial S_1} = \partial_n \phi |_{\partial S_2} \Rightarrow \int d\vec{x}\cdot\hat{n} [\partial_n \phi |_{\partial S_1} = \partial_n \phi |_{\partial S_2}]\ = \phi |_{\partial S_1} = \phi |_{\partial S_2}$$

Thus, the requirement in 1D1V amounts to selecting densities $f_e, f_i$ that satisfy the neutrality condition:

$$\int dx \rho (x) = \int dx \left[ \int dv \left( f_i (x,v) - f_e(x,v)\right)\right] = 0$$

or

$$\int dx \int dv f_i(x,v) = \int dx \int dv f_e (x,v)$$

In general, 

$$\langle n_i \rangle = \langle n_e \rangle$$

This is the solubility condition that can also be seen numerically. Here, we repeat a section from the notebook <a href = "https://github.com/dsirajud/IPython-notebooks/blob/master/DECSKS-04%20--%20Design%20of%20a%206th%20order%20FD%20Poisson%20solver/DECSKS-04%20--%20Design%20of%20a%206th%20order%20FD%20Poisson%20solver.ipynb">DECSKS-04</a> which motivated the FD Poisson solver:

<hr>

## "
### Consequences of periodicity and how to choose compatible densities $n_e$ and $n_i$

This has been stated above in the development noting that the matrix $\underline{\underline{D}}$ is circulant, but to wit consider the general multidimensional case:

$$\Delta \phi = g$$

where $g \equiv n_e - n_i$ is the source function. Numerically, we discretize the above to arrive at the equation

$$D\phi = b$$

where $D$ is a a discretization of the multidimensional Laplacian operator with accompanying Dirichlet boundary and $g$ is discretized straightforwardly. Note, that $A$ is positive definite, hence zero is not an eigenvalue and the matrix is nonsingular. Thus, there exists a solution for any $g$.

Now, consider $D$ as the disretization of the Laplacian withi periodic boundary. It was shown above this matrix is circulant, hence zero is an eigenvalue (matrix is positive <i>semi</i> definite), and the matrix is not invertible. There exists a kernel $K$ with solutions $\phi \equiv C = \text{const}$ belonging to the column space of $D$. Thus, $\text{range } D = K^{\perp}$ and $D\phi = g$ has no solution unless

$$1\cdot g = 0, \qquad \text{solubility condition}$$, 

where $1 \equiv \underline{1}$ is the vector of all ones. Numerically, this statement is the solubility condition for the periodic Poisson problem. Analytically, it is perhaps easier to see. Consider $\phi \in C^2 (\Omega )$, we have

$$\Delta \phi = g$$

integrating over the domain, 

$$\int_{\Omega} \Delta \phi dV = \int_{\Omega} g dV$$


the left-hand side becomes

$$\int_{\Omega} \Delta \phi dV = \int_{\partial\Omega}\frac{\partial \phi}{\partial\vec{n}}dS = 0, \qquad \text{for a periodic boundary on } \phi$$

Thus, we have

$$0 = \int_{\Omega} g dV$$

(This is the same as $1\cdot g = 0$ in the numerical discussion above, i.e. the average value of $g$ must be zero).

In 1D over $x\in [a,b]$, for a source $g = n_e - n_i$ we have

$$0 = \int_a^b dx (n_e(x) - n_i(x))$$

#### Case 1: suppose $n_e = n_e(x)$ is chosen, $n_i(x) = \text{const}$ to be found consistently for periodic BCs

The above gives

$$0 = \int_a^b dx n_e(x) - \int_a^b dx n_i$$

or

$$(b - a)n_i = \int_a^b dx n_e(x) \quad \Rightarrow \quad \boxed{n_i = \frac{\int_a^b dx n_e(x)}{b - a}}\qquad (1) $$

#### Case 2: suppose $n_e = n_e(x)$ is chosen, $n_i = n_i(x)$ to be selected consistently for periodic BCs

It is easy to see the above requirement amounts to:

$$\int_a^b dx n_i (x) = \int_a^b dx n_e(x)$$

or

$$\langle n_i \rangle_x = \langle n_e \rangle_x$$

Which restates what was just said, the average values must be hte same (so that the source $\langle g\rangle  = \langle n_e - n_i\rangle = 0$. Hence, it does not matter the shape of the functions so long as averages are the same.

## "

<hr>


We reproduce the cold background case as a starting point vs. the electron density with a bump on its otherwise Maxwellian tail:


$$f_e(0,x,v)  = \frac{1}{\sqrt{2\pi}}(1 + 0.04\cos (0.3 x))(0.9 e^{-v^2 / 2} + 0.2 e^{-4 (v - 4.5)^2})$$

over the domain $x\in [a_x, b_x] = [-2\pi / 0.3 , 2\pi / 0.3 ]$, $v\in [a_v, b_v] = [-8.0, 8.0]$. Over the finite domain, we can integrate analytically and fall back on error functions to numerically obtain the results.

$$n_e(0,x) = \int_{a_v}^{b_v} dv f_e(0,x,v) = \frac{1}{\sqrt{2\pi}}(1 + 0.04\cos (0.3 x))\int_{a_v}^{b_v} dv(0.9 e^{-v^2 / 2} + 0.2 e^{-4 (v - 4.5)^2})$$

Recalling the error function definition

$$\text{erf}(z) = \frac{2}{\sqrt{\pi}}\int_0^z e^{-t^2} dt$$ 

The first integral:

\begin{eqnarray*}
    \int_{a_v}^{b_v} dv e^{-v^2 / 2} & = & \int_{a_v/\sqrt{2}}^{b_v/\sqrt{2}} \sqrt{2}du e^{-u^2} = -\sqrt{2}\int_{0}^{a_v/\sqrt{2}}du e^{-u^2}  + \sqrt{2}\int_{0}^{b_v/\sqrt{2}}  du e^{-u^2} \\
& = & \sqrt{2}\left(\frac{\sqrt{\pi}}{2}\right)\left(-\text{erf}(a_v) + \text{erf}(b_v / \sqrt{2})\right)　\\
& = & \sqrt{\frac{\pi}{2}}\left(-\text{erf}(a_v / \sqrt{2}) + \text{erf}(b_v / \sqrt{2})\right)
\end{eqnarray*}

Here, we have $a_v = -8.0, b_v = 8.0$ or $a_v = -b_v$, 

$$\int_{a_v}^{b_v} dv e^{-v^2 / 2} = \sqrt{\frac{\pi}{2}}\left(-\text{erf}(-b_v/ \sqrt{2}) + \text{erf}(b_v/ \sqrt{2})\right)$$

By examining the definition of the $\text{erf}$ function we can see that $\text{erf}(-z) = -\text{erf}(z)$, thus the above is

$$\int_{a_v}^{b_v} dv e^{-v^2 / 2} = \sqrt{\frac{\pi}{2}}\left(2\text{erf}(b_v / \sqrt{2})\right) = \sqrt{2\pi}\text{erf}(b_v/ \sqrt{2}) = \sqrt{2\pi}\text{erf}(8.0 / \sqrt{2}) = (0.9999999999999987558)\sqrt{2\pi}$$

or 

$$\int_{a_v}^{b_v} dv e^{-v^2 / 2} \simeq 2.50663$$




The second integral:

$$\int_{a_v}^{b_v} dv e^{-4 (v - 4.5)^2} = \int_{2(a_v - 4.5)}^{2(b_v - 4.5)} \frac{du}{2} e^{-u^2}$$

which can be written as:

\begin{eqnarray*}
\int_{2(a_v - 4.5)}^{2(b_v - 4.5)} \frac{du}{2} e^{-u^2} & = & -\int_0^{2(a_v - 4.5)}\frac{du}{2} e^{-u^2} + \int_{0}^{2(b_v - 4.5)} \frac{du}{2} e^{-u^2} \\
& = & -\frac{\sqrt{\pi}}{4}\text{erf}(2(a_v - 4.5)) + \frac{\sqrt{\pi}}{4}\text{erf}(2(b_v - 4.5))\\
& = & -\frac{\sqrt{\pi}}{4}\text{erf}(2(-8.0 - 4.5)) + \frac{\sqrt{\pi}}{4}\text{erf}(2(8.0 - 4.5))\\
& = & -\frac{\sqrt{\pi}}{4}(-1.0000000000000000000000000000000000000000000000000)  \\
&&\phantom{-\frac{\sqrt{\pi}}{4}}+ \frac{\sqrt{\pi}}{4}(0.9999999999999999999999582)\\ 
& = & \frac{\sqrt{\pi}}{4}(1.0 + 0.9999999999999999999999582)\\
& = & \frac{\sqrt{\pi}}{4}(1.9999999999999999999999582)\\
\int_{a_v}^{b_v} dv e^{-4 (v - 4.5)^2} = & \simeq & 0.886227
\end{eqnarray*}'

Thus, the density is

$$n_e(0,x) = \int_{a_v}^{b_v} dv f_e(0,x,v) = \frac{1}{\sqrt{2\pi}}(1 + 0.04\cos (0.3 x))\int_{a_v}^{b_v} dv(0.9 e^{-v^2 / 2} + 0.2 e^{-4 (v - 4.5)^2})$$

or

$$n_e(0,x) = \frac{1}{\sqrt{2\pi}}(1 + 0.04\cos (0.3 x))[(0.9)(2.50663)+ 0.2(0.886227)] $$

or

$$n_e(0,x) = \frac{\text{const}}{\sqrt{2\pi}}(1 + 0.04\cos (0.3 x))$$

where

$$\text{const} = [(0.9)(2.50663)+ 0.2(0.886227)]= 2.4332124$$


Integrating over the spatial coordinate is trivial.

\begin{eqnarray*}
\int_{a_x}^{b_x} dx n_e(0,x) & = & \frac{\text{const}}{\sqrt{2\pi}}\cdot \int_{a_x}^{b_x} dx (1 + 0.04 \cos (0.3 x)) \\
& = & \frac{\text{const}}{\sqrt{2\pi}}\cdot \left[x + \frac{1}{0.3}\sin (0.3 x)\right]_{a_x}^{b_x} \\
&= &\frac{\text{const}}{\sqrt{2\pi}}\cdot \left[ (b_x - a_x) + \frac{1}{0.3}(\sin(0.3 b_x) - \sin (0.3 a_x))\right]
\end{eqnarray*}


Recalling $a_x = -2\pi / 0.3$, $b_x = 2\pi / 0.3$ or $a_x = -b_x$, we have

\begin{eqnarray*}
\int_{a_x}^{b_x} dx n_e(0,x) & = & \frac{\text{const}}{\sqrt{2\pi}}\cdot \left[ 2b_x + \frac{1}{0.3}2\sin(0.3 b_x)\right] \\
& = & \frac{\text{const}}{\sqrt{2\pi}}\cdot \left[ 2(2\pi / 0.3) + \frac{1}{0.3}2\sin(0.3 ((2\pi / 0.3))\right] \\ 
& = & \frac{\text{const}}{\sqrt{2\pi}}\cdot \left[ 4\pi / 0.3 + 0\right]  \\
& = & \frac{2.4332124}{\sqrt{2\pi}}\cdot \frac{4\pi}{0.3} \\
\int_{a_x}^{b_x} dx n_e(0,x)  & =  & 40.6611
\end{eqnarray*}

If we want a cold ion density, then we determine the density according to eq. (1) above:

$$\langle n_i \rangle = \frac{\int_{a_x}^{b_x} dx n_e }{b_x - a_x} = \frac{40.6611}{2\pi / 0.3 - (-2\pi / 0.3)} = 0.970712$$

This ensures quasineutrality, but as discussed is a more fundamental (pure math) condition that must be met ($\langle n_i \rangle = \langle n_e \rangle$) so that periodic BCs on the potentially are actually true. This density has been used before, even in the legacy version DECSKS-1.2, so we know it is the proper selection for $n_i$.



To put this in a code that evolves two distribution functions $f_e$ and $f_i$, we require consistently loading this "cold background" into $f_i$ so that its average gives the above value. That is,

$$f_i(x,v) = \frac{\langle n_i\rangle}{b_v - a_v} \qquad \forall x, v$$

so that

$$\int_{a_x}^{b_x} dx \int_{a_v}^{b_v}dv f_i(x,v) = \int_{a_x}^{b_x} dx \int_{a_v}^{b_v}dv \frac{\langle n_i\rangle}{b_v - a_v} = \langle n_i\rangle\int_{a_x}^{b_x} dx \int_{a_v}^{b_v}dv  \frac{1}{b_v - a_v}$$

or

$$\langle n_i\rangle (b_x - a_x) = \int_{a_x}^{b_x} dx n_e(x)$$

which is the solubility condition, thus such a density selection for $n_i$ is acceptable.

Performing a simulation for this "cold background ion density" in DECSKS-2.0 which actually evolves two densities $f_i$ and $f_e$, we should recover mostly the same result because of the ion inertia will prevent significant movement or acceleration from energy exchanges with the electric field over the short duration of our simulation. The simulation ran is <a href = "./DECSKS/v2.0/etc/params-s17-01.dat/">etc/params_s17-01.dat</a>

In [ ]:
""